In [1]:
import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

####### 修改输入文件
file = h5py.File('data/fused_100_96.h5','r')
imageData   = file['imageData'][:]
imageLabel  = file['imageLabel'][:]  
file.close()

# 随机打乱数据和标签
N = imageData.shape[0]
index = np.random.permutation(N)
data  = imageData[index,:,:]
label = imageLabel[index]

# 对数据升维,标签one-hot
data  = np.expand_dims(data, axis=1)
label = convert_to_one_hot(label,49).T
label = np.argmax(label, axis=1)  

# 划分数据集
N = data.shape[0]
num_train = round(N*0.8)
X_train = data[0:num_train,:,:,:]
Y_train = label[0:num_train]
X_test  = data[num_train:N,:,:,:]
Y_test  = label[num_train:N]

print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

/home/data_mining/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


X_train shape: (19162, 1, 100, 96)
Y_train shape: (19162,)
X_test shape: (4791, 1, 100, 96)
Y_test shape: (4791,)


In [2]:
class dataset_prediction(Dataset):
    def __init__(self, data_features, data_target):
        self.len = len(data_features)
        self.features = torch.from_numpy(data_features)
        self.target = torch.from_numpy(data_target)
        
    def __getitem__(self, index):
        return self.features[index], self.target[index]

    def __len__(self):
        return self.len

train_set = dataset_prediction(data_features=X_train, data_target=Y_train)
test_set = dataset_prediction(data_features=X_test, data_target=Y_test)

train_loader = DataLoader(dataset=train_set,
                            batch_size=64,
                            shuffle=True,
                            drop_last=True)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelAttentionModule(nn.Module):
    def __init__(self, channel, reduction=16):  
        super(ChannelAttentionModule, self).__init__()
        mid_channel = channel // reduction
        # 使用自适应池化缩减map的大小，保持通道不变  
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.shared_MLP = nn.Sequential(
            nn.Linear(in_features=channel, out_features=mid_channel),
            nn.ReLU(),
            nn.Linear(in_features=mid_channel, out_features=channel)
        )
        self.sigmoid = nn.Sigmoid()
        # self.act=SiLU()
    
    def forward(self, x):
        avgout = self.shared_MLP(self.avg_pool(x).view(x.size(0),-1)).unsqueeze(2).unsqueeze(3)
        maxout = self.shared_MLP(self.max_pool(x).view(x.size(0),-1)).unsqueeze(2).unsqueeze(3)
        return self.sigmoid(avgout + maxout)
        
# 空间注意力模块
class SpatialAttentionModule(nn.Module):
    def __init__(self):
        super(SpatialAttentionModule, self).__init__()
        self.conv2d = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3) 
        # self.act=SiLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # map尺寸不变，缩减通道
        avgout = torch.mean(x, dim=1, keepdim=True)
        maxout, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avgout, maxout], dim=1)
        out = self.sigmoid(self.conv2d(out))
        return out

# CBAM模块
class CBAM(nn.Module):
    def __init__(self, channel): 
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttentionModule(channel)
        self.spatial_attention = SpatialAttentionModule()

    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out

# Example usage for image classification
class CNN_CBAM(nn.Module):
    def __init__(self):
        super(CNN_CBAM, self).__init__()
        
        self.conv1 = nn.Sequential(
            # 输入[1,200,60]
            nn.Conv2d(
                in_channels=1,
                out_channels=32,
                kernel_size=(20,3),
                stride=1,
                padding=2,
            ),
            # 经过卷积层 输出[32,200,60] 传入池化层
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(10,1))   # 经过池化 输出[32,20,60] 传入下一个卷积
        )
        ## 第二层卷积
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,    # 同上
                out_channels=64,
                kernel_size=(3,3),
                stride=1,
                padding=1
            ),
            # 经过卷积 输出[64, 40, 30] 传入池化层
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(3,2))  # 经过池化 输出[64,7,30] 传入下一个卷积
        )
        ## 第三层卷积
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,    # 同上
                out_channels=128,
                kernel_size=(3,3),
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.cbam = CBAM(channel=128)
        self.classifier = nn.Linear(128*50, 49)  # Define your classifier

    def forward(self, x):
        # x: batch_size x channels x height x width (e.g., an image)
        x = torch.tensor(x)
        x = x.to(torch.float32)
        x = x.cuda()
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x) 
        x = self.cbam(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)  # Classification
        return x

In [6]:
# cnn 实例化
cnn_cbam = CNN_CBAM()
print(cnn_cbam)

if torch.cuda.is_available():
    cnn_cbam = cnn_cbam.cuda()

epoches = 100
learning_rate = 0.001

# 定义优化器和损失函数
optimizer = torch.optim.Adam(cnn_cbam.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_function = loss_function.cuda()

# 开始训练
for epoch in range(epoches):
    print("进行第{}个epoch".format(epoch))
    for step, (batch_x, batch_y) in enumerate(train_loader):
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        
        output = cnn_cbam(batch_x)  # batch_x=[64,1,200,60]

        loss = loss_function(output, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 为了实时显示准确率
        if step % 64 == 0:
            test_output = cnn_cbam(X_test).cpu()
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float(np.sum(pred_y == Y_test)) / float(Y_test.shape[0])
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.cpu().data.numpy(), '| test accuracy: %.2f' % accuracy)


test_output = cnn_cbam(X_test[:10]).cpu()
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(pred_y)
print(X_test[:10])


CNN_CBAM(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(20, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(10, 1), stride=(10, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(3, 2), stride=(3, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cbam): CBAM(
    (channel_attention): ChannelAttentionModule(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (max_pool): AdaptiveMaxPool2d(output_size=1)
      (shared_MLP): Sequential(
        (0): Linear(in_features=128, out_features=8, bias=True)
        (1): ReLU()
        (2): Linear(in_features=8, out_features=128, bias=True)
      )
      

/home/data_mining/.local/lib/python3.6/site-packages/ipykernel_launcher.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch:  0 | train loss: 3.8921 | test accuracy: 0.04
Epoch:  0 | train loss: 1.7664 | test accuracy: 0.44
Epoch:  0 | train loss: 1.3026 | test accuracy: 0.64
Epoch:  0 | train loss: 0.7281 | test accuracy: 0.74
Epoch:  0 | train loss: 0.7793 | test accuracy: 0.80
进行第1个epoch
Epoch:  1 | train loss: 0.5463 | test accuracy: 0.80
Epoch:  1 | train loss: 0.4133 | test accuracy: 0.83
Epoch:  1 | train loss: 0.5936 | test accuracy: 0.84
Epoch:  1 | train loss: 0.5188 | test accuracy: 0.84
Epoch:  1 | train loss: 0.3467 | test accuracy: 0.87
进行第2个epoch
Epoch:  2 | train loss: 0.3131 | test accuracy: 0.85
Epoch:  2 | train loss: 0.4153 | test accuracy: 0.87
Epoch:  2 | train loss: 0.2412 | test accuracy: 0.88
Epoch:  2 | train loss: 0.4484 | test accuracy: 0.89
Epoch:  2 | train loss: 0.4891 | test accuracy: 0.87
进行第3个epoch
Epoch:  3 | train loss: 0.1614 | test accuracy: 0.89
Epoch:  3 | train loss: 0.2204 | test accuracy: 0.89
Epoch:  3 | train loss: 0.1562 | test accuracy: 0.90
Epoch:  3 | t